# Lab 10: Demography

In lecture 10.1 we discussed how we can apply what we know about linear multivariate models to populations structured by stage or age (demography). We looked at a few simple examples, starting from the transition matrices. Here we'll see how to build the transition matrix from data, and how to analyze a larger transition matrix with the help of Sage.

## The demography of Canadians

Here we'll use data from past censuses to build a Leslie matrix and analyze how the demography of Canadians is expected to change. 

Because the census takes place every 5 years, instead of tracking individuals of each age, we'll track individuals in 5-year incremements: 0-4 years, 5-9 years, .... This ensures all individuals in a given age class will either die or move to the next age class, i.e., we get a Leslie matrix. We'll also group all individuals over 85 into the final age class, which means that we have a bit of a modified Leslie matrix with a non-zero survival probability in the bottom right corner. Because these individuals do not reproduce, this actually doesn't complicate things much.

We'll track just the number of females in each age class for simplicity, so we'll have to divide the number of offspring by the sex ratio (fraction female) to get the expected number of female offspring.

First, let's see how many females were in each age class in 1991 (which we'll take to be a representative time) 

In [ ]:
age = ['%d-%d' %(i,i+4) for i in range(0,85,5)] + ['85+'] #list of age classes (just to make nicer tables)
n = [945400, 909700, 901500, 895900, 976800, 1168800, 1221800, 1133000, 1031500, 807500, 656600, 610100, 599400, 571800, 461700, 362600, 237900, 197100] #number of females in each age class
table(columns=[n], header_column=age)

In [ ]:
bar_chart(n, axes_labels=['age class', 'number of females'])

Now, let's figure out the survival probabilities for each age class, the $p_i$'s of our Leslie matrix.

To do this let's use the number of females of each age class that died that year.

In [ ]:
d = [1344, 155, 152, 345, 361, 556, 681, 939, 1337, 1659, 2142, 3275, 5032, 7763, 9757, 12949, 14261, 27415]
table(columns=[d], header_column=age)

In [ ]:
bar_chart(d, axes_labels=['age class', 'female death rate per year'])

### Q1. [1 point] Use the total number of females in each age class, $n_i$, and the number of female deaths in each age class per year, $d_i$, to calaculate the per year probability of dying in each age class, $d_i/n_i$. The probability of surviving between one census and the next is then $p_i = (1 - d_i/n_i)^5$. Calculate this for each age class, making a list called ```p```. Make a bar chart if you'd like to better see the pattern.

The remaining part of the Leslie matrix is the fecundity, the $m_i$'s. To calculate these we use the number of births by age class in 1991.

In [ ]:
b = [0, 0, 265, 24180, 80723, 150024, 107560, 33107, 4124, 138, 0, 0, 0, 0, 0, 0, 0, 0]
table(columns=[b], header_column=age)

In [ ]:
bar_chart(b, axes_labels=['age class', 'births per year'])

### Q2. [1 point] To get the number of births per year per female in each age class, we first divide the number of births by the number of females, $b_i/n_i$. To get the number of daughters we then multiply this by the sex ratio at birth, $0.4867$. Then, to get the birth rate per census in each class, $m_i$, we multiply each of these by 5. Calculate the $m_i$'s, making a list called ```m```. Make a bar plot if you want (it should have the same shape as the one above -- a good check!).

OK, now we have all the elements of our Leslie matrix. Putting things together is a bit tricky, but here is one way to do it (try writing a script yourself if you'd like -- I'm sure there are better ways to do it!)

In [ ]:
s = len(p) #number of age classes
L = matrix(RDF,s,s,0) #real (RR) matrix, size sxs, filled with 0s
L[0] = m #add fecundities in the first row of L
for i,pi in enumerate(p[:-1]): #for all but the final age class...
    L[i+1,i] = pi #put the survival probability in row i+1, column i of L
L[-1,-1] = p[-1] #add the probability of remaining in the final class as the bottom right entry in L
L #take a look!

### Q3. [2 points] Now, what is the expected long-term population growth rate given this Leslie matrix? Is the population expected to grow or decline? Given the Canadian population has grown, what are we missing? 

Note that you should get the (approximately) same answer using the Euler-Lotka equation, $1 = \sum_{i=0}^{s}\frac{l_i m_i}{\lambda^i}$, which we can do without having to make the Leslie matrix

In [ ]:
l = [prod(p[:i]).n() for i in range(len(p))] #probability of surviving to age class i
var('R')
find_root(sum(li * mi / R^(i+1) for i,(li,mi) in enumerate(zip(l,m))) - 1, 0, 1) #find the real positive roots

### Q4. [1 point] What is the predicted frequency of each age(-class) distribution in the long-run? Make a bar chart to see this visually (set ymin=0 to see the pattern better).

### Q5. [1 point] What are the reproductive values of each age class relative to the reproductive value of the first age class? Make a bar chart.

### Q6. [2 points] Use equation 10.21, $\frac{\mathrm{d}\lambda}{\mathrm{d}p_i} = \frac{u_i v_{i+1}}{\vec{v}^T \vec{u}}$, to calculate the sensitivities of the long-term growth rate to the survival probabilities of each age class. Based on this, increasing the survival probability of what age class would have the largest effect?

### Q7. [2 points] Use equation 10.22, $\frac{\mathrm{d}\lambda}{\mathrm{d}m_i} = \frac{u_i v_1}{\vec{v}^T \vec{u}}$, to calculate the sensitivities of the long-term growth rate to the fecundities of each age class. Based on this, increasing the fecundity of what age class would have the largest effect? (the answer is weird! bonus: why?)

### Q8. [1 point] Use the Leslie matrix to iterate the population forward to 2021. Make a bar chart of the number of females in each age class. What age class do we expect to be most freuent?